In [1]:
%matplotlib inline
import torch
import numpy as np
import timeit
import matplotlib.pyplot as plt
import scipy
from scipy import interpolate

In [2]:
# midpoint
def computeParticlePathMid(X0,Vfun,h,n):

    X = torch.zeros(2,n+1)
    X[:,0] = X0
    t = 0
    for i in range(n):
        # Change this for interpolation
        V = Vfun(X[:,i],t)
        
        if i==0:
            X[:,i+1] = X[:,i] + h*V
        else:
            X[:,i+1] = X[:,i-1] + 2*h*V
        t +=h
    return X

In [3]:
def RBFfun(x,y,xI,yI,sigma):
    phi = torch.exp(-((x-xI)**2 + (y-yI)**2)/sigma**2)
    return phi

In [4]:
def RBFmatrix(x,y,sigma):

    n = x.shape[0]
    A = torch.zeros(n,n);
    for i in range(n):
        for j in range(n):
            A[i,j] = RBFfun(x[i],y[i],x[j],y[j],sigma);
    return A

In [5]:
def RBFinterp(x,y,f,xI,yI,sigma):

    # Compute the RBF matrix
    A = RBFmatrix(x,y,sigma);

    # Compute the coefficients
    c, LU = torch.solve(f.unsqueeze(1),A)

    # Compute the data on the new points
    nI = xI.shape[0]
    n  = x.shape[0]
    vI = torch.zeros(nI)
    for i in range(nI):
        for j in range(n):
            vi = c[j]*RBFfun(x[j],y[j],xI[i],yI[i],sigma)
            vI[i] += vi[0];
    
    return vI

In [ ]:
def getVelFromObs(xout,yout,xobs,yobs,uobs,vobs):
    